In [1]:
import numpy as np



In [2]:
import tensorflow as tf


In [3]:
from sklearn.model_selection import train_test_split


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [5]:
from sklearn.linear_model import PassiveAggressiveClassifier


In [6]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [7]:
import pandas as pd


In [8]:
import itertools

In [9]:
fake=pd.read_csv(r"G:\machine learning\dataset\fake_news/fake.csv")


In [10]:
true=pd.read_csv(r"G:\machine learning\dataset\true_news/true.csv")

In [11]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [12]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [13]:
fake.shape ,true.shape

((23481, 4), (21417, 4))

In [14]:
fake["class"]=0
true["class"]=1

In [15]:
merge=pd.concat([fake,true],axis=0)

In [16]:
merge.head(15)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [17]:
data=merge.drop(["subject","date","title"],axis=1)
data.head(15)

,text,class
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0
5,The number of cases of cops brutalizing and ki...,0
6,Donald Trump spent a good portion of his day a...,0
7,In the wake of yet another court decision that...,0
8,Many people have raised the alarm regarding th...,0
9,Just when you might have thought we d get a br...,0


In [18]:
data=data.sample(frac=1)
data.head(10)

,text,class
6168,WASHINGTON (Reuters) - President Donald Trump ...,1
20064,This woman should be in a jail cell not sittin...,0
20013,Say what??? When former Clinton cheerleader Ma...,0
14515,ISLAMABAD (Reuters) - Pakistani authorities is...,1
889,"WASHINGTON (Reuters) - Sam Clovis, a top advis...",1
20792,WASHINGTON/BEIJING (Reuters) - President Donal...,1
10991,COLBERT LANGUAGE WARNING! This late night ho...,0
9757,Andrea Mitchell is annoyed with President Tr...,0
21598,It doesn t get much more anti-American than fo...,0
7745,A group of House Republicans are so determined...,0


In [19]:
def word_drop(text):
    text=text.lower()
    text=re.sub('\[.*?\]','',text)
    text=re.sub("\\W"," ",text)
    text=re.sub('https?://\S+|www\.\S+','',text)
    text=re.sub('<.*?>+','',text)
    text=re.sub('[%s]'% re.escape(string.punctuation),'',text)
    text=re.sub('\n','',text)
    text=re.sub('\w*\d\w*','',text)
    return text

In [20]:
import string
import re
data["text"]=data["text"].apply(word_drop)

In [21]:
data.head(15)

,text,class
6168,washington reuters president donald trump ...,1
20064,this woman should be in a jail cell not sittin...,0
20013,say what when former clinton cheerleader ma...,0
14515,islamabad reuters pakistani authorities is...,1
889,washington reuters sam clovis a top advis...,1
20792,washington beijing reuters president donal...,1
10991,colbert language warning this late night ho...,0
9757,andrea mitchell is annoyed with president tr...,0
21598,it doesn t get much more anti american than fo...,0
7745,a group of house republicans are so determined...,0


In [22]:
x=data["text"]
y=data['class']


In [23]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
x_train,x_test ,y_train,y_test=train_test_split(x,y,test_size=0.25)

In [24]:
vectorization =  TfidfVectorizer()
 
xv_train=vectorization.fit_transform(x_train)
xv_test=vectorization.transform(x_test)


Logistic Regression


In [126]:
from sklearn.linear_model import LogisticRegression

In [127]:
LR=LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [128]:
LR.score(xv_train,y_train)

0.9909422979835476

In [129]:
pred_LR=LR.predict(xv_test)

In [130]:
print(classification_report(y_test,pred_LR))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5848
           1       0.99      0.99      0.99      5377

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [131]:
from sklearn.tree import DecisionTreeClassifier
DT=DecisionTreeClassifier()

In [132]:
clf_gini = DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=0)

In [133]:
DT.fit(xv_train,y_train)

DecisionTreeClassifier()

In [134]:
model_DT=DT.predict(xv_test)

In [135]:
print(classification_report(y_test,model_DT))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5848
           1       1.00      1.00      1.00      5377

    accuracy                           1.00     11225
   macro avg       1.00      1.00      1.00     11225
weighted avg       1.00      1.00      1.00     11225



In [136]:
from sklearn.ensemble import GradientBoostingClassifier

In [137]:
GRC=GradientBoostingClassifier(random_state=0)
GRC.fit(xv_train,y_train)


GradientBoostingClassifier(random_state=0)

In [138]:
GRC.score(xv_test,y_test)

0.9955456570155902

In [139]:
pred_GRC=GRC.predict(xv_test)

In [140]:
print(classification_report(y_test,pred_GRC))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5848
           1       0.99      1.00      1.00      5377

    accuracy                           1.00     11225
   macro avg       1.00      1.00      1.00     11225
weighted avg       1.00      1.00      1.00     11225



Random forest classifier

In [141]:
from sklearn.ensemble import RandomForestClassifier

In [142]:
RFC=RandomForestClassifier(random_state=0)
RFC.fit(xv_train,y_train)

RandomForestClassifier(random_state=0)

In [143]:
RFC.score(xv_test,y_test)

0.9910022271714922

In [144]:
pred_RFC=RFC.predict(xv_test)

In [145]:
print(classification_report(y_test,pred_RFC))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5848
           1       0.99      0.99      0.99      5377

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [154]:
def output_label(n):
        if n==0:
            return "Fake News"
        elif n==1:
            return "True News"
        
        
def testing(news):
    testing_news={"text":[news]}
    new_def_test=pd.DataFrame(testing_news)
    new_def_test["text"]=new_def_test["text"].apply(word_drop)
    new_x_test=new_def_test["text"]
    new_xv_test=vectorization.transform(new_x_test)
    pred_LR=LR.predict(new_xv_test)
    pred_DT=DT.predict(new_xv_test)
    pred_GRC=GRC.predict(new_xv_test)
    pred_RFC=RFC.predict(new_xv_test)
    return print("\n\nLR Prediction: {} \nDTPrediction: {}  \nGBC Prediction: {}  \nRFC Prediction: {}".format(output_label(pred_LR),
                                                                                                               output_label(pred_DT),
                                                                                                               output_label(pred_GRC),
                                                                                                               output_label(pred_RFC)))

In [155]:
news=str(input())
testing(news)

Researchers looked at 40,000 marriages that took place in rural India between 1960 and 2008.  They found that dowry was paid in 95% of the marriages even though it's been illegal in India since 1961.  The practice, often described as a social evil, continues to thrive and leaves women vulnerable to domestic violence and even death.  Paying and accepting dowry is a centuries-old tradition in South Asia where the bride's parents gift cash, clothes and jewellery to the groom's family.  The study was based on dowry data from 17 Indian states that contain 96% of India's population. It focussed on rural India since a majority of Indians continue to live in villages.  Economists S Anukriti, Nishith Prakash and Sunghoh Kwon used information on value of gifts - cash and kind - received or given at the time of marriage.


LR Prediction: Fake News 
DTPrediction: Fake News  
GBC Prediction: Fake News  
RFC Prediction: Fake News
